In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import torch
import torch.nn as nn
from torchvision import models, transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.optim import Adam
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [ ]:
base_dir = '/content/drive/MyDrive/chest_xray/chest_xray'
train_dir = base_dir + '/train'
validation_dir = base_dir + '/val'
test_dir = base_dir + '/test'

# ImageNet normalization stats
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),  # small shifts
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(root=validation_dir, transform=val_test_transforms)
test_dataset = datasets.ImageFolder(root=test_dir, transform=val_test_transforms)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}, Test samples: {len(test_dataset)}")


Train samples: 5216, Val samples: 16, Test samples: 624


In [ ]:
from collections import Counter
import torch

def get_targets_from_dataset(ds):
    # Handle Subset(ImageFolder(...)) or plain ImageFolder
    if hasattr(ds, 'dataset') and hasattr(ds, 'indices'):
        base = ds.dataset
        idxs = ds.indices
        if hasattr(base, 'targets'):
            return [base.targets[i] for i in idxs]
        elif hasattr(base, 'samples'):
            return [base.samples[i][1] for i in idxs]
    else:
        if hasattr(ds, 'targets'):
            return list(ds.targets)
        elif hasattr(ds, 'samples'):
            return [s[1] for s in ds.samples]
    # Fallback (slow)
    return [label for _, label in ds]

targets = get_targets_from_dataset(train_loader.dataset)
class_counts = Counter(targets)
num_classes = len(class_counts)

# Inverse-frequency class weights (higher = rarer class)
class_weights = torch.zeros(num_classes, dtype=torch.float)
total = len(targets)
for cls, cnt in class_counts.items():
    class_weights[cls] = total / (num_classes * cnt)

class_weights = class_weights.to(device)
print("Class counts:", class_counts)
print("Class weights:", class_weights)


Class counts: Counter({1: 3875, 0: 1341})
Class weights: tensor([1.9448, 0.6730])


In [ ]:
import numpy as np
from torch.utils.data import WeightedRandomSampler, DataLoader

# Per-sample weights: pick class weight for each sample’s label
sample_weights = np.array([class_weights[label].item() for label in targets], dtype=np.float64)

sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

# Rebuild train_loader with the sampler (do NOT use shuffle with sampler)
train_loader = DataLoader(
    train_loader.dataset,
    batch_size=train_loader.batch_size if hasattr(train_loader, "batch_size") else 32,
    sampler=sampler,
    num_workers=train_loader.num_workers if hasattr(train_loader, "num_workers") else 2,
    pin_memory=True
)
print("Rebuilt train_loader with WeightedRandomSampler.")


Rebuilt train_loader with WeightedRandomSampler.


In [ ]:
model=models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
for param in model.parameters():
  param.requires_grad= False
num_features = model.fc.in_features  # number of inputs to the FC layer
model.fc = nn.Linear(num_features, 2)  # new FC layer for 2 output classes

model=model.to(device)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 323MB/s]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import recall_score

drive_path = '/content/drive/MyDrive/chest_xray/best_model.pth'

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = nn.functional.cross_entropy(inputs, targets, weight=self.alpha, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

criterion = FocalLoss(alpha=class_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30
patience = 5
best_val_recall = 0
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    train_loss, correct, total = 0, 0, 0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    train_acc = 100. * correct / total

    model.eval()
    val_loss, correct, total = 0, 0, 0
    all_targets, all_preds = [], []
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            all_targets.extend(targets.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
    val_acc = 100. * correct / total
    val_recall = recall_score(all_targets, all_preds, average='macro')

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss/len(train_loader):.4f} | "
          f"Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.2f}% | Macro Recall: {val_recall:.3f}")

    if val_recall > best_val_recall:
        best_val_recall = val_recall
        torch.save({
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_macro_recall': val_recall
        }, drive_path)
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break


Epoch 1/30 | Train Loss: 0.1207 | Val Loss: 0.1009 | Val Acc: 93.75% | Macro Recall: 0.938
Epoch 2/30 | Train Loss: 0.0753 | Val Loss: 0.1634 | Val Acc: 93.75% | Macro Recall: 0.938
Epoch 3/30 | Train Loss: 0.0666 | Val Loss: 0.0703 | Val Acc: 87.50% | Macro Recall: 0.875
Epoch 4/30 | Train Loss: 0.0653 | Val Loss: 0.1601 | Val Acc: 93.75% | Macro Recall: 0.938
Epoch 5/30 | Train Loss: 0.0670 | Val Loss: 0.1420 | Val Acc: 93.75% | Macro Recall: 0.938
Epoch 6/30 | Train Loss: 0.0564 | Val Loss: 0.1041 | Val Acc: 100.00% | Macro Recall: 1.000
Epoch 7/30 | Train Loss: 0.0609 | Val Loss: 0.0833 | Val Acc: 93.75% | Macro Recall: 0.938
Epoch 8/30 | Train Loss: 0.0587 | Val Loss: 0.0553 | Val Acc: 93.75% | Macro Recall: 0.938
Epoch 9/30 | Train Loss: 0.0597 | Val Loss: 0.1482 | Val Acc: 87.50% | Macro Recall: 0.875
Epoch 10/30 | Train Loss: 0.0539 | Val Loss: 0.0764 | Val Acc: 100.00% | Macro Recall: 1.000
Epoch 11/30 | Train Loss: 0.0569 | Val Loss: 0.0489 | Val Acc: 87.50% | Macro Recall: 0

In [ ]:
import torch
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.001)
drive_path = "/content/drive/MyDrive/chest_xray/best_model.pth"

checkpoint = torch.load(drive_path, map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
epoch = checkpoint["epoch"]
best_val_recall = checkpoint["val_macro_recall"]

print(f"Model restored from epoch {epoch}, achieving a validation macro recall of {best_val_recall:.3f}")
model.eval()


Model restored from epoch 6, achieving a validation macro recall of 1.000


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())


print(classification_report(all_labels, all_preds, target_names=['NORMAL', 'PNEUMONIA']))
print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


              precision    recall  f1-score   support

      NORMAL       0.83      0.90      0.86       234
   PNEUMONIA       0.94      0.89      0.91       390

    accuracy                           0.89       624
   macro avg       0.88      0.89      0.89       624
weighted avg       0.90      0.89      0.89       624

Confusion Matrix:
[[211  23]
 [ 44 346]]


In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from sklearn.metrics import accuracy_score, recall_score, classification_report, confusion_matrix

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
for p in model.parameters():
    p.requires_grad = False
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)
model = model.to(device)

checkpoint = torch.load(drive_path, map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
epoch = checkpoint["epoch"]
best_val_recall = checkpoint["val_macro_recall"]
print(f"Best checkpoint: epoch {epoch}, val macro recall {best_val_recall:.3f}")

model.eval()
all_true, all_pred = [], []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        preds = torch.argmax(logits, dim=1)
        all_true.extend(y.cpu().numpy())
        all_pred.extend(preds.cpu().numpy())

acc = accuracy_score(all_true, all_pred)
macro_recall = recall_score(all_true, all_pred, average="macro")
report = classification_report(all_true, all_pred, target_names=["NORMAL","PNEUMONIA"])
cm = confusion_matrix(all_true, all_pred, labels=[0,1])

print(f"Test Acc: {acc*100:.2f}% | Test Macro Recall: {macro_recall:.3f}")
print(report)
print("Confusion Matrix:")
print(cm)


Best checkpoint: epoch 6, val macro recall 1.000
Test Acc: 89.26% | Test Macro Recall: 0.894
              precision    recall  f1-score   support

      NORMAL       0.83      0.90      0.86       234
   PNEUMONIA       0.94      0.89      0.91       390

    accuracy                           0.89       624
   macro avg       0.88      0.89      0.89       624
weighted avg       0.90      0.89      0.89       624

Confusion Matrix:
[[211  23]
 [ 44 346]]
